In [1]:
import os
import sys
import glob
import shutil
import pandas as pd
import numpy as np
from ipywidgets import interact
from copy import deepcopy
import matplotlib.pyplot as plt
print('OK')

OK


In [2]:
sys.path.append('..')

In [3]:
from dataset.dataset.dsindex import FilesIndex
from preprocessing.ct_masked_batch import CTImagesMaskedBatch as CTIMB

### Global index

In [4]:
DIR_LUNA = '/notebooks/data/MRT/luna/s*/*.mhd'
DIR_DUMP = '/notebooks/Koryagin/Batch_dev/dir_for_dumps/'

In [5]:
if os.path.exists(DIR_DUMP):
    shutil.rmtree(DIR_DUMP)

In [6]:
ind = FilesIndex(path=DIR_LUNA, no_ext=True)

### Create batch by subindexing

In [7]:
batch_masked = CTIMB(ind.create_subset(ind.index[[714, 297, 299, 672]]))
batch_masked_2 = CTIMB(batch_masked.index)

### Test actions

* check load

In [8]:
batch_masked.load(fmt='raw')
batch_masked_2.load(fmt='raw')

* load info about nodules in batches

In [9]:
nodules = pd.read_csv('/notebooks/data/MRT/luna/CSVFILES/annotations.csv')
nodules.head(2)

,seriesuid,coordX,coordY,coordZ,diameter_mm
0,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,-128.699421,-175.319272,-298.387506,5.651471
1,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,103.783651,-211.925149,-227.121250,4.224708


In [10]:
batch_masked = batch_masked.fetch_nodules_info(nodules_df=nodules)
batch_masked_2 = batch_masked_2.fetch_nodules_info(nodules_df=nodules)

* create mask in one of two batches before spacing unification

In [11]:
batch_masked.create_mask()

* check unify_spacing

In [12]:
batch_masked.unify_spacing(shape=(384, 448, 448), spacing=(0.9, 0.9, 0.9))

/usr/local/lib/python3.5/dist-packages/scipy/ndimage/interpolation.py:600: UserWarning: From scipy 0.13.0, the output shape of zoom() is calculated with round() instead of int() - for these inputs the size of the returned array has changed.
  "the returned array has changed.", UserWarning)


In [13]:
batch_masked_2.unify_spacing(shape=(384, 448, 448), spacing=(0.9, 0.9, 0.9))

/usr/local/lib/python3.5/dist-packages/scipy/ndimage/interpolation.py:600: UserWarning: From scipy 0.13.0, the output shape of zoom() is calculated with round() instead of int() - for these inputs the size of the returned array has changed.
  "the returned array has changed.", UserWarning)


* create mask in the second batch after spacing unification

In [14]:
batch_masked_2.create_mask()

* Dump first batch (all components) with unified spacing

In [15]:
batch_masked.dump(dst=DIR_DUMP)

selector_events.py[LINE:53]#DEBUG    [2017-07-11 13:20:02,272]  Using selector: EpollSelector


* See what's been dumped

In [16]:
os.listdir(os.path.join(DIR_DUMP, batch_masked.indices[0]))

['masks.blk', 'shape.pkl', 'images.blk', 'spacing.pkl', 'origin.pkl']

### Now create new batch and load everything in there (from dump)

In [17]:
batch_from_dump = CTIMB(FilesIndex(path=DIR_DUMP + '/*', dirs=True))

In [18]:
from datetime import datetime

In [19]:
now = datetime.now()
batch_from_dump.load(fmt='blosc', src_blosc=['images', 'spacing', 'origin', 'masks'])
new_now = datetime.now()
print('7 pats, load: ', new_now - now)


7 pats, load:  0:00:14.230237


### Check the whole thing (by comparing three batches)

In [20]:
def plot_arr_slices(height, *arrays, clim=(-1200, 300)):
    fig, axes = plt.subplots(1, len(arrays), figsize=(14, len(arrays)*8))
    
    for arr, i in zip(arrays, range(len(arrays))):
        depth = arr.shape[0]
        n_slice = int(depth * height)
        
        kwargs = dict()
        if np.max(arr) - np.min(arr) > 2.0:
            kwargs.update(clim=clim)
        else:
            kwargs.update(clim=(0, 1))
    
        axes[i].imshow(arr[n_slice], cmap=plt.cm.gray, **kwargs)
    plt.show()

In [21]:
n_pat = 0

In [22]:
ind_npat = batch_masked.indices[1]

In [23]:
interact(lambda height: plot_arr_slices(height, 
                                        batch_masked.get(ind_npat, 'images'),
                                        batch_masked_2.get(ind_npat, 'masks'),
                                        batch_from_dump.get(ind_npat, 'masks')),
         height=(0.01, 0.99, 0.01))



<function __main__.<lambda>>